In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

yr2020_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv')
yr2020_df['fips'] = yr2020_df['fips'].fillna(0).astype(np.int64)
yr2020_df['date'] = pd.to_datetime(yr2020_df['date'], yearfirst=True)
yr2021_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv')
yr2021_df['fips'] = yr2021_df['fips'].fillna(0).astype(np.int64)
yr2021_df['date'] = pd.to_datetime(yr2021_df['date'], yearfirst=True)
yr2022_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2022.csv')
yr2022_df['fips'] = yr2022_df['fips'].fillna(0).astype(np.int64)
yr2022_df['date'] = pd.to_datetime(yr2022_df['date'], yearfirst=True)
us_covid_df = pd.concat([yr2020_df, yr2021_df, yr2022_df])
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2544371,2022-06-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2544372,2022-06-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2544373,2022-06-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2544374,2022-07-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
65091,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
65093,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
65095,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
65097,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
65099,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1537929,2022-06-28,Arkansas,Arkansas,5001,5861,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1537931,2022-06-29,Arkansas,Arkansas,5001,5867,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1537933,2022-06-30,Arkansas,Arkansas,5001,5872,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1537935,2022-07-01,Arkansas,Arkansas,5001,5877,78.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2544371,2022-06-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2544372,2022-06-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2544373,2022-06-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2544374,2022-07-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-07-02') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
893,2022-07-02,Snohomish,Washington,174852,1193.00,53061,WA,County,822083,21269.39,145.12
1784,2022-07-02,Cook,Illinois,1287748,15085.00,17031,IL,County,5150233,25003.68,292.90
2674,2022-07-02,Orange,California,654017,7126.00,6059,CA,County,3175692,20594.47,224.39
3563,2022-07-02,Maricopa,Arizona,1345286,17495.00,4013,AZ,County,4485414,29992.46,390.04
4452,2022-07-02,Los Angeles,California,3126003,32344.00,6037,CA,County,10039107,31138.26,322.18
...,...,...,...,...,...,...,...,...,...,...,...
2541987,2022-07-02,Wheeler,Oregon,254,3.00,41069,OR,County,1332,19069.07,225.23
2542615,2022-07-02,King,Texas,52,0.00,48269,TX,County,272,19117.65,0.00
2543212,2022-07-02,Esmeralda,Nevada,110,3.00,32009,NV,County,873,12600.23,343.64
2543805,2022-07-02,Loving,Texas,231,1.00,48301,TX,County,169,136686.39,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
893,2022-07-02,Snohomish,Washington,174852,1193.00,53061,WA,County,822083,21269.39,145.12,145.12,21269.39
1784,2022-07-02,Cook,Illinois,1287748,15085.00,17031,IL,County,5150233,25003.68,292.90,292.90,25003.68
2674,2022-07-02,Orange,California,654017,7126.00,6059,CA,County,3175692,20594.47,224.39,224.39,20594.47
3563,2022-07-02,Maricopa,Arizona,1345286,17495.00,4013,AZ,County,4485414,29992.46,390.04,390.04,29992.46
4452,2022-07-02,Los Angeles,California,3126003,32344.00,6037,CA,County,10039107,31138.26,322.18,322.18,31138.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541987,2022-07-02,Wheeler,Oregon,254,3.00,41069,OR,County,1332,19069.07,225.23,225.23,19069.07
2542615,2022-07-02,King,Texas,52,0.00,48269,TX,County,272,19117.65,0.00,0.00,19117.65
2543212,2022-07-02,Esmeralda,Nevada,110,3.00,32009,NV,County,873,12600.23,343.64,343.64,12600.23
2543805,2022-07-02,Loving,Texas,231,1.00,48301,TX,County,169,136686.39,591.72,591.72,136686.39


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1608970,2022-07-02,Hale,Alabama,4969,106.00,1065,AL,County,14651,33915.77,723.50,723.50,33915.77,1
1356714,2022-07-02,Winston,Alabama,7790,133.00,1133,AL,County,23629,32967.96,562.87,562.87,32967.96,2
931596,2022-07-02,Franklin,Alabama,10115,140.00,1059,AL,County,31362,32252.41,446.40,446.40,32252.41,3
1108677,2022-07-02,Clay,Alabama,4226,82.00,1027,AL,County,13235,31930.49,619.57,619.57,31930.49,4
1002314,2022-07-02,Morgan,Alabama,37498,519.00,1103,AL,County,119679,31332.15,433.66,433.66,31332.15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352574,2022-07-02,Johnson,Wyoming,1965,20.00,56019,WY,County,8445,23268.21,236.83,236.83,23268.21,19
2174052,2022-07-02,Big Horn,Wyoming,2699,63.00,56003,WY,County,11790,22892.28,534.35,534.35,22892.28,20
1913557,2022-07-02,Lincoln,Wyoming,4317,32.00,56023,WY,County,19830,21770.05,161.37,161.37,21770.05,21
1535459,2022-07-02,Sublette,Wyoming,2004,27.00,56035,WY,County,9831,20384.50,274.64,274.64,20384.50,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1255581,2022-07-02,Lowndes,Alabama,2723,77.00,1085,AL,County,9726,27997.12,791.69,791.69,27997.12,31,1
1253923,2022-07-02,Crenshaw,Alabama,3969,102.00,1041,AL,County,13772,28819.34,740.63,740.63,28819.34,22,2
1608970,2022-07-02,Hale,Alabama,4969,106.00,1065,AL,County,14651,33915.77,723.50,723.50,33915.77,1,3
530162,2022-07-02,Walker,Alabama,19363,449.00,1127,AL,County,63521,30482.83,706.85,706.85,30482.83,10,4
1106187,2022-07-02,Butler,Alabama,5255,129.00,1013,AL,County,19448,27020.77,663.31,663.31,27020.77,38,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352574,2022-07-02,Johnson,Wyoming,1965,20.00,56019,WY,County,8445,23268.21,236.83,236.83,23268.21,19,19
1865118,2022-07-02,Uinta,Wyoming,5815,39.00,56041,WY,County,20226,28750.12,192.82,192.82,28750.12,8,20
1913557,2022-07-02,Lincoln,Wyoming,4317,32.00,56023,WY,County,19830,21770.05,161.37,161.37,21770.05,21,21
1250607,2022-07-02,Albany,Wyoming,10481,51.00,56001,WY,County,38880,26957.30,131.17,131.17,26957.30,13,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,145.12,21269.39,32,16
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,145.12,21269.39,32,16
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,145.12,21269.39,32,16
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,145.12,21269.39,32,16
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,145.12,21269.39,32,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538519,2022-06-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2538520,2022-06-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2538521,2022-06-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2538522,2022-07-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1602294,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,33915.77,3,1,1.00,nan
1602295,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,33915.77,3,1,0.00,nan
1602296,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,33915.77,3,1,0.00,nan
1602297,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,33915.77,3,1,0.00,nan
1602298,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,723.50,33915.77,3,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946241,2022-06-28,Crook,Wyoming,1390,29.00,56011,WY,County,7584,18328.06,382.38,382.38,18328.06,10,23,5.00,0.93
1946242,2022-06-29,Crook,Wyoming,1390,29.00,56011,WY,County,7584,18328.06,382.38,382.38,18328.06,10,23,0.00,0.93
1946243,2022-06-30,Crook,Wyoming,1390,29.00,56011,WY,County,7584,18328.06,382.38,382.38,18328.06,10,23,0.00,0.93
1946244,2022-07-01,Crook,Wyoming,1390,29.00,56011,WY,County,7584,18328.06,382.38,382.38,18328.06,10,23,0.00,0.93


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa['14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa['deaths'] = usa['deaths'].astype(int)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1250555,2020-03-26,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27997.12,1,31,1.00,137.07,0.00,nan
1250556,2020-03-27,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27997.12,1,31,0.00,118.43,0.00,nan
1250557,2020-03-28,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27997.12,1,31,0.00,106.71,0.00,nan
1250558,2020-03-29,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27997.12,1,31,0.00,98.00,0.00,nan
1250559,2020-03-30,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27997.12,1,31,0.00,88.21,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524282,2022-06-28,Teton,Wyoming,10807,16,56039,WY,County,23464,46057.79,68.19,68.19,46057.79,23,1,108.00,18.07,0.00,0.00
524283,2022-06-29,Teton,Wyoming,10807,16,56039,WY,County,23464,46057.79,68.19,68.19,46057.79,23,1,0.00,18.07,0.00,0.00
524284,2022-06-30,Teton,Wyoming,10807,16,56039,WY,County,23464,46057.79,68.19,68.19,46057.79,23,1,0.00,18.07,0.00,0.00
524285,2022-07-01,Teton,Wyoming,10807,16,56039,WY,County,23464,46057.79,68.19,68.19,46057.79,23,1,0.00,18.07,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-07-02') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
633794,2022-07-02,Imperial,California,70676,943,6025,CA,County,181215,39001.19,520.38,520.38,39001.19,1,1,0.00,128.57,0.00,0.00
96444,2022-07-02,Shasta,California,38811,656,6089,CA,County,180080,21552.09,364.28,364.28,21552.09,2,33,0.00,56.50,0.00,0.00
313900,2022-07-02,San Bernardino,California,631190,7860,6071,CA,County,2180085,28952.54,360.54,360.54,28952.54,3,7,0.00,917.79,0.00,6.79
1536215,2022-07-02,Tuolumne,California,14940,189,6109,CA,County,54478,27423.91,346.93,346.93,27423.91,4,10,0.00,33.14,0.00,0.00
4452,2022-07-02,Los Angeles,California,3126003,32344,6037,CA,County,10039107,31138.26,322.18,322.18,31138.26,5,4,0.00,4878.21,0.00,6.71
173310,2022-07-02,Tulare,California,143769,1500,6107,CA,County,466195,30838.81,321.75,321.75,30838.81,6,5,0.00,212.86,0.00,0.29
1864192,2022-07-02,Tehama,California,15404,205,6103,CA,County,65084,23667.88,314.98,314.98,23667.88,7,25,0.00,18.21,0.00,0.00
1115269,2022-07-02,Inyo,California,4944,56,6027,CA,County,18039,27407.28,310.44,310.44,27407.28,8,11,0.00,7.57,0.00,0.00
139553,2022-07-02,Stanislaus,California,144885,1694,6099,CA,County,550660,26311.15,307.63,307.63,26311.15,9,15,0.00,213.43,0.00,0.00
1009749,2022-07-02,Merced,California,75900,832,6047,CA,County,277680,27333.62,299.63,299.63,27333.62,10,12,0.00,126.71,0.00,0.36


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
633794,2022-07-02,Imperial,California,70676,943,6025,CA,County,181215,39001.19,520.38,520.38,39001.19,1,1,0.00,128.57,0.00,0.00
1355000,2022-07-02,Kings,California,58078,449,6031,CA,County,152940,37974.37,293.58,293.58,37974.37,12,2,0.00,76.79,0.00,0.07
2391127,2022-07-02,Lassen,California,11113,70,6035,CA,County,30573,36349.07,228.96,228.96,36349.07,21,3,0.00,6.57,0.00,0.07
4452,2022-07-02,Los Angeles,California,3126003,32344,6037,CA,County,10039107,31138.26,322.18,322.18,31138.26,5,4,0.00,4878.21,0.00,6.71
173310,2022-07-02,Tulare,California,143769,1500,6107,CA,County,466195,30838.81,321.75,321.75,30838.81,6,5,0.00,212.86,0.00,0.29
63394,2022-07-02,Madera,California,46546,372,6039,CA,County,157327,29585.51,236.45,236.45,29585.51,19,6,0.00,85.64,0.00,0.00
313900,2022-07-02,San Bernardino,California,631190,7860,6071,CA,County,2180085,28952.54,360.54,360.54,28952.54,3,7,0.00,917.79,0.00,6.79
395325,2022-07-02,Kern,California,257276,2460,6029,CA,County,900202,28579.81,273.27,273.27,28579.81,14,8,0.00,294.36,0.00,0.29
62546,2022-07-02,Fresno,California,274259,2783,6019,CA,County,999101,27450.58,278.55,278.55,27450.58,13,9,0.00,469.50,0.00,1.07
1536215,2022-07-02,Tuolumne,California,14940,189,6109,CA,County,54478,27423.91,346.93,346.93,27423.91,4,10,0.00,33.14,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
633794,2022-07-02,Imperial,California,70676,943,6025,CA,County,181215,39001.19,520.38,520.38,39001.19,1,1,0.00,128.57,0.00,0.00
96444,2022-07-02,Shasta,California,38811,656,6089,CA,County,180080,21552.09,364.28,364.28,21552.09,2,33,0.00,56.50,0.00,0.00
313900,2022-07-02,San Bernardino,California,631190,7860,6071,CA,County,2180085,28952.54,360.54,360.54,28952.54,3,7,0.00,917.79,0.00,6.79
1536215,2022-07-02,Tuolumne,California,14940,189,6109,CA,County,54478,27423.91,346.93,346.93,27423.91,4,10,0.00,33.14,0.00,0.00
4452,2022-07-02,Los Angeles,California,3126003,32344,6037,CA,County,10039107,31138.26,322.18,322.18,31138.26,5,4,0.00,4878.21,0.00,6.71
173310,2022-07-02,Tulare,California,143769,1500,6107,CA,County,466195,30838.81,321.75,321.75,30838.81,6,5,0.00,212.86,0.00,0.29
1864192,2022-07-02,Tehama,California,15404,205,6103,CA,County,65084,23667.88,314.98,314.98,23667.88,7,25,0.00,18.21,0.00,0.00
1115269,2022-07-02,Inyo,California,4944,56,6027,CA,County,18039,27407.28,310.44,310.44,27407.28,8,11,0.00,7.57,0.00,0.00
139553,2022-07-02,Stanislaus,California,144885,1694,6099,CA,County,550660,26311.15,307.63,307.63,26311.15,9,15,0.00,213.43,0.00,0.00
1009749,2022-07-02,Merced,California,75900,832,6047,CA,County,277680,27333.62,299.63,299.63,27333.62,10,12,0.00,126.71,0.00,0.36


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'case_day', '14day_case_avg']]
top10_ca_case_df['case_day'] = top10_ca_case_df['case_day'].astype(int)
top10_ca_case_df['date'] =  pd.to_datetime(top10_ca_case_df['date'], infer_datetime_format=True)
top10_ca_case_df

,date,county,cases,current_crate,case_day,14day_case_avg
0,2020-03-20,Imperial,4,2.21,4,1.86
1,2020-03-21,Imperial,4,2.21,0,1.86
2,2020-03-22,Imperial,4,2.21,0,1.79
3,2020-03-23,Imperial,4,2.21,0,1.79
4,2020-03-24,Imperial,9,4.97,5,1.79
...,...,...,...,...,...,...
8361,2022-06-28,Lassen,11097,36296.73,22,7.43
8362,2022-06-29,Lassen,11106,36326.17,9,7.36
8363,2022-06-30,Lassen,11113,36349.07,7,7.36
8364,2022-07-01,Lassen,11113,36349.07,0,6.57


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'death_day', '14day_death_avg']]
top10_ca_death_df['death_day'] = top10_ca_death_df['death_day'].astype(int)
top10_ca_death_df['date'] =  pd.to_datetime(top10_ca_death_df['date'], infer_datetime_format=True)
top10_ca_death_df

,date,county,deaths,current_drate,death_day,14day_death_avg
0,2020-03-20,Imperial,0,0.00,0,0.00
1,2020-03-21,Imperial,0,0.00,0,0.00
2,2020-03-22,Imperial,0,0.00,0,0.00
3,2020-03-23,Imperial,0,0.00,0,0.00
4,2020-03-24,Imperial,0,0.00,0,0.00
...,...,...,...,...,...,...
8400,2022-06-28,Merced,830,298.91,2,0.36
8401,2022-06-29,Merced,830,298.91,0,0.36
8402,2022-06-30,Merced,830,298.91,0,0.36
8403,2022-07-01,Merced,832,299.63,2,0.36


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)